In [322]:
import radd
from radd import build
from radd import fit
from scipy.stats.mstats import mquantiles as mq
from scipy.stats.mstats_extras import mjci
from radd.tools import theta 

homedir = os.path.expanduser('~')
cox = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/Coxon/coxon_behav_data_binned.csv"))
#cox = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/Coxon/coxon_behav_data.csv"))
pro = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/SS/Proactive/pro_data.csv"))
re = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/SS/Reactive/Re_Data.csv"), index_col=0)
ssb = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/SSBayes/alldata_clean.csv"), index_col=0)

re_idx = re.idx.unique().tolist()
idx_to_fit = re_idx[:5]
re_small = re[re.idx.isin(idx_to_fit)]

In [323]:
from radd import models
models = reload(models)
build=reload(build)
from radd import CORE
CORE = reload(CORE)
from radd import fit
fit = reload(fit)

In [324]:
p = {'a': 0.4, 'v': 1.5, 'xb': 1.1, 'ssv': -1.2, 'tr': 0.15}
#p = {'a': 0.57168, 'xb': 0.91224, 'ssv': -1.35826, 'tr': 0.1137, 'v': 1.163}
#p = {'a': 0.426849, 'xb': 0.98831, 'ssv': -0.895592,  'tr': 0.166207, 'v': 0.98773}
# 'v_bsl': 0.98670, 'v_pnl': 0.94318
#p = {'a': 0.5878,  'ssv': -1.1539, 'tr': 0.120, 'v': 0.95904, 'xb': 1.845}

In [408]:
#popt_flat = {'a': 0.5878,  'ssv': -1.1539, 'tr': 0.120, 'v': 0.95904, 'xb': 1.845}
y_flat = array([ 0.93463,  0.99385,  0.98402,  0.91107,  0.54959,  0.14221,
         0.51458,  0.5458 ,  0.56643,  0.58621,  0.614  ,  0.49743,
         0.52929,  0.55066,  0.56961,  0.59699])
yhat_flat = array([ 0.96565,  1.     ,  0.99799,  0.92021,  0.55117,  0.16872,
        0.51673,  0.5451 ,  0.56654,  0.58796,  0.61784,  0.50442,
        0.5306 ,  0.54872,  0.56774,  0.59491])
finfo_flat_dict = {'logp': -134.9745683471391, 'df': 11, 'ssv': -1.1539911430640575, 'xb': 1.845200842716557, 'nfev': 117, 'nvary': 5, 'AIC': -124.9745683471391, 'a': 0.5878677077625838, 'chi': 0.0023862904091628626, 'tr': 0.12069921450345845, 'BIC': -130.59254171246522, 'rchi': 0.00021693549174207842, 'cnvrg': True, 'v': 0.9590383185377456, 'ndata': 16}
# finfo_flat = pd.Series(finfo_flat_dict)

#v_bsl = 0.98670
#v_pnl = 0.94318
#popt['v'] = np.array([v_bsl, v_pnl])
y = array([ 0.94201,  0.99426,  0.98279,  0.89672,  0.50492,  0.10328,
        0.51263,  0.54257,  0.56297,  0.58212,  0.61031,  0.49795,
        0.52919,  0.54817,  0.56757,  0.59428,  0.92725,  0.99344,
        0.98525,  0.92541,  0.59426,  0.18115,  0.51652,  0.54902,
        0.56989,  0.59031,  0.6177 ,  0.49691,  0.52939,  0.55316,
        0.57166,  0.59969])
yhat = array([ 0.97924,  1.     ,  0.99715,  0.89649,  0.49095,  0.12952,
        0.51026,  0.53772,  0.55901,  0.58015,  0.61188,  0.4994 ,
        0.52516,  0.54306,  0.56173,  0.58938,  0.95521,  0.99999,
        0.99841,  0.93175,  0.59041,  0.19477,  0.51974,  0.54848,
        0.57081,  0.59186,  0.62216,  0.50768,  0.53345,  0.55241,
        0.5712 ,  0.59826])

finfo_dict = {'a': 0.5878677368164062, 'nvary': 2, 'logp': -279.2829727203045, 'df': 30, 'chi': 0.004861561891504873, 'ssv': -1.1539911031723022, 'tr': 0.1206992119550705, 'BIC': -275.1240896369448, 'xb': 1.8452008962631226, 'v_bsl': 0.9867001692803977, 'nfev': 55, 'cnvrg': True, 'v_pnl': 0.9431834622518617, 'ndata': 32, 'rchi': 0.00016205206305016243, 'AIC': -275.2829727203045}
# finfo = pd.Series(finfo_dict)

In [410]:
fp = dict(deepcopy(m.fitparams))

def breakdown_vector(vector, fp):
    nssd = fp['nssd']+1
    nquant = len(fp['percentiles'])
    
    presp = vector[1:nssd]
    quant = vector[-nquant*2:]
    quant_cor = quant[:nquant]
    quant_err = quant[-nquant:]
    #quant_err = vector[nssd+nquant:]
    
    return presp, quant_cor, quant_err


phat, qchat, qehat = [], [], []
for vector in m.yhat_flat_list:
    presp, quant_cor, quant_err = breakdown_vector(vector, fp)
    phat.append(presp)
    qchat.append(quant_cor)
    qehat.append(quant_err)

p, qc, qe = [], [], []
for vector in m.observed_flat:
    presp, quant_cor, quant_err = breakdown_vector(vector, fp)
    p.append(presp)
    qc.append(quant_cor)
    qe.append(quant_err)

In [409]:
plt.plot(y)
plt.plot(yhat)


In [325]:
m = build.Model(data=re, inits=p, depends_on={'v': 'Cond'}, kind='xdpm', fit_on='average')
m.set_basinparams(nrand_inits=30)
m.opt.basinparams

{'disp': True,
 'interval': 10,
 'maxiter': 100,
 'method': 'TNC',
 'niter': 40,
 'niter_success': 20,
 'nrand_inits': 30,
 'nrand_samples': 5000,
 'stepsize': 0.05,
 'tol': 0.001}

In [326]:
m.optimize()

default inits: fmin0=1.253468037
basinhopping step 0: f 0.00323848
basinhopping step 1: f 0.002267 trial_f 0.002267 accepted 1  lowest_f 0.002267
found new global minimum on step 1 with function value 0.002267
basinhopping step 2: f 0.0232627 trial_f 0.0232627 accepted 1  lowest_f 0.002267
basinhopping step 3: f 0.0439561 trial_f 0.0439561 accepted 1  lowest_f 0.002267
basinhopping step 4: f 0.00885709 trial_f 0.00885709 accepted 1  lowest_f 0.002267
basinhopping step 5: f 0.0226364 trial_f 0.0226364 accepted 1  lowest_f 0.002267
basinhopping step 6: f 0.0226364 trial_f 0.203284 accepted 0  lowest_f 0.002267
basinhopping step 7: f 0.0979609 trial_f 0.0979609 accepted 1  lowest_f 0.002267
basinhopping step 8: f 0.0209808 trial_f 0.0209808 accepted 1  lowest_f 0.002267
basinhopping step 9: f 0.0211941 trial_f 0.0211941 accepted 1  lowest_f 0.002267
adaptive stepsize: acceptance rate 0.800000 target 0.500000 new stepsize 0.0555556 old stepsize 0.05
basinhopping step 10: f 0.12902 trial_f 

In [407]:
m.observed_flat#.flatten()

[array([ 0.93463,  0.99385,  0.98402,  0.91107,  0.54959,  0.14221,
         0.51458,  0.5458 ,  0.56643,  0.58621,  0.614  ,  0.49743,
         0.52929,  0.55066,  0.56961,  0.59699])]

In [393]:
m.popt_flat_list[0]

{'a': 0.5878677368164062,
 'ssv': -1.1539911031723022,
 'tr': 0.1206992119550705,
 'v': array([ 0.95904]),
 'xb': 1.8452008962631226}

In [394]:
m.popt_list[0]

{'a': 0.58786774,
 'ssv': -1.1539911,
 'tr': 0.12069921,
 'v': array([ 0.9867 ,  0.94318]),
 'v_bsl': 0.98670016928039772,
 'v_pnl': 0.94318346225186167,
 'xb': 1.8452009}

In [399]:
dict(m.finfo_list[0])

{'AIC': -275.2829727203045,
 'BIC': -275.1240896369448,
 'a': 0.5878677368164062,
 'chi': 0.0048615618915048729,
 'cnvrg': True,
 'df': 30,
 'logp': -279.2829727203045,
 'ndata': 32,
 'nfev': 55,
 'nvary': 2,
 'rchi': 0.00016205206305016243,
 'ssv': -1.1539911031723022,
 'tr': 0.1206992119550705,
 'v_bsl': 0.98670016928039772,
 'v_pnl': 0.94318346225186167,
 'xb': 1.8452008962631226}

In [400]:
finfo_flat_dict = {'logp': -134.9745683471391, 'df': 11, 'ssv': -1.1539911430640575, 'xb': 1.845200842716557, 'nfev': 117, 'nvary': 5, 'AIC': -124.9745683471391, 'a': 0.5878677077625838, 'chi': 0.0023862904091628626, 'tr': 0.12069921450345845, 'BIC': -130.59254171246522, 'rchi': 0.00021693549174207842, 'cnvrg': True, 'v': 0.9590383185377456, 'ndata': 16}
finfo_flat = pd.Series(finfo_flat_dict)

finfo_dict = {'a': 0.5878677368164062, 'nvary': 2, 'logp': -279.2829727203045, 'df': 30, 'chi': 0.004861561891504873, 'ssv': -1.1539911031723022, 'tr': 0.1206992119550705, 'BIC': -275.1240896369448, 'xb': 1.8452008962631226, 'v_bsl': 0.9867001692803977, 'nfev': 55, 'cnvrg': True, 'v_pnl': 0.9431834622518617, 'ndata': 32, 'rchi': 0.00016205206305016243, 'AIC': -275.2829727203045}
finfo = pd.Series(finfo_dict)

In [ ]:
dict

In [384]:
import numpy
#import numpy as np
numpy = reload(numpy)
import pandas as pd

#pd=reload(pd)


In [386]:
pd.__version__

u'0.18.0'

In [363]:
par = params.tolist()
[par.remove(parameter) for parameter in list(m.pc_map)]

[None]

In [362]:
m.pc_map.keys()

['v']

In [358]:
fitdf_cols

['idx',
 'a',
 'ssv',
 'tr',
 'v',
 'xb',
 'v_bsl',
 'v_pnl',
 'nfev',
 'nvary',
 'df',
 'chi',
 'rchi',
 'logp',
 'AIC',
 'BIC',
 'cnvrg']

In [355]:
np.squeeze(m.pc_map.values())


array(['v_bsl', 'v_pnl'], 
      dtype='|S5')

In [344]:
h.make_dataframes()

In [345]:
m.fitinfo

,idx,Cond,a,ssv,tr,v,xb,nfev,nvary,df,chi,rchi,logp,AIC,BIC,cnvrg
0,28,bsl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28,pnl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29,bsl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29,pnl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30,bsl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,30,pnl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,31,bsl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,31,pnl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,32,bsl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,32,pnl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [334]:
fcopy

,idx,Cond,a,ssv,tr,v,xb,nfev,nvary,df,chi,rchi,logp,AIC,BIC,cnvrg
0,NaN,NaN,0.58787,-1.154,0.1207,NaN,1.8452,55,2,30,0.0048616,0.00016205,-279.28,-275.28,-275.12,True
1,28,pnl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29,bsl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29,pnl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30,bsl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,30,pnl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,31,bsl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,31,pnl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,32,bsl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,32,pnl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [316]:



yhat = array([0.9835, 1.0, 0.9993, 0.9235, 0.4862, 0.1535, 0.5075, 0.5343, 0.5544, 0.5762, 0.6078, 0.4995, 0.5240, 0.5436, 0.5622, 0.5912, 0.9593, 1.0, 0.9996, 0.9487, 0.5821, 0.2071, 0.5181, 0.5477, 0.5683, 0.5901, 0.6217, 0.5091, 0.5345, 0.5532, 0.5741, 0.6029])

y = array([0.9420, 0.9942, 0.9827, 0.8967, 0.5049, 0.1032, 0.5126, 0.5425, 0.5629, 0.5821, 0.6103, 0.4979, 0.5291, 0.5481, 0.5675, 0.5942, 0.9272, 0.9934, 0.9852, 0.9254, 0.5942, 0.1811, 0.5165, 0.5490, 0.5698, 0.5903, 0.6176, 0.4969, 0.5293, 0.5531, 0.5716, 0.5996])

plt.plot(yhat)
plt.plot(y)











In [239]:
m.set_basinparams(tol=.001, nrand_samples=200, disp=True)


y = m.observed[0]
wts = m.cost_wts[0]
fp = m.set_fitparams(idx=0, y=y, wts=wts, get_params=True)


In [176]:
y = m.observed_flat[0]
wts = m.flat_cost_wts[0]
fp = m.set_fitparams(idx=0, y=y, wts=wts, get_params=True)

In [ ]:
 = m.opt.run_basinhopping(p=p, is_flat=False)

In [190]:
basin_keys = np.sort(list(m.pc_map))
nlevels = fp['y'].ndim
basin_params = deepcopy(p)
xmin, xmax = theta.format_basinhopping_bounds(basin_keys, nlevels=nlevels)

In [192]:
x = np.hstack(np.hstack([basin_params[pk]*np.ones(nlevels) for pk in basin_keys])).tolist()

In [193]:
x

[0.9877302444240045, 0.9877302444240045]

In [191]:
xmin

[0.1, 0.1]

In [145]:
m.opt.simulator.pp

{'a': 0.4268497874206143,
 'ssv': -0.8955923693264308,
 'tr': 0.16620719971331227,
 'v': array([ 1.1 ,  1.01]),
 'xb': 0.9883206351695517}

In [88]:
len(basin_keys)

1

In [206]:
xopt

array([ 0.98773,  0.98773])

In [89]:

xoptx = array([xopt]).reshape(len(basin_keys), nlevels)








ValueError: total size of new array must be unchanged

In [80]:
xopt

array([[ 0.98773,  0.98773]])

In [85]:
m.make_optimizer(nrand_inits=10, method='Nelder-Mead', bmethod='TNC', bdisp=True, tol=1e-3, btol=1e-2)

In [133]:
m.optimize(maxfev=100, tol=1.e-3, btol=1.e-2, multiopt=True, fit_flat=False, fit_cond=True, bdisp=False)

Optimization terminated successfully.
         Current function value: 0.000014
         Iterations: 14
         Function evaluations: 32
True


In [136]:
m.yhat_flat_list

yhat = array([0.9707, 1.0, 0.9978, 0.903, 0.4987, 0.1323, 0.5033, 0.5352, 0.5563, 0.5803, 0.6136, 0.4913, 0.5193, 0.5389, 0.5588, 0.5888, 0.9402, 1.0, 0.9989, 0.9395, 0.6067, 0.2040, 0.5151, 0.5464, 0.5701, 0.5922, 0.6237, 0.5004, 0.5289, 0.5491, 0.5698, 0.5980])

y = array([0.9420, 0.9942, 0.9827, 0.8967, 0.5049, 0.1032, 0.5126, 0.5425, 0.5629, 0.5821, 0.6103, 0.4979, 0.5291, 0.5481, 0.5675, 0.5942, 0.9272, 0.9934, 0.9852, 0.9254, 0.5942, 0.1811, 0.5165, 0.5490, 0.5698, 0.5903, 0.6176, 0.4969, 0.5293, 0.5531, 0.5716, 0.5996])






In [89]:
m.fitparams['y']

array([ 0.93463,  0.99385,  0.98402,  0.91107,  0.54959,  0.14221,
        0.51458,  0.5458 ,  0.56643,  0.58621,  0.614  ,  0.49743,
        0.52929,  0.55066,  0.56961,  0.59699])

In [144]:
from radd.tools import vis
from sklearn.neighbors import KernelDensity

In [150]:
def plot_kde_cdf(quant, bw=.1, ax=None, color=None):

    if ax is None:
        f, ax = plt.subplots(1, figsize=(5, 5))
    if color is None:
        color = 'k'
    kdefits = kde_fit_quantiles(quant, bw=bw)
    sns.kdeplot(kdefits, cumulative=True,  color=color, ax=ax, linewidth=2.5)

    ax.set_xlim(kdefits.min() * .94, kdefits.max() * 1.05)
    ax.set_ylabel('P(RT<t)')
    ax.set_xlabel('RT (s)')
    ax.set_ylim(-.05, 1.05)
    ax.set_xticklabels(ax.get_xticks() * .1)

    plt.tight_layout()
    sns.despine()

def kde_fit_quantiles(rtquants, nsamples=1000, bw=.1):
    """ takes quantile estimates and fits cumulative density function
    returns samples to pass to sns.kdeplot()
    """
    kdefit = KernelDensity(kernel='gaussian', bandwidth=bw).fit(rtquants)
    samples = kdefit.sample(n_samples=nsamples).flatten()
    return samples


def unpack_yvector(y, kind=''):

    sc, gq, eq = y[1:6], y[6:11], y[11:]

    return sc, gq, eq


In [151]:
sc, gq, eq = unpack_yvector(y, kind=m.kind)
fitsc, fitgq, fiteq = unpack_yvector(yhat, kind=m.kind)
kdefits = [kde_fit_quantiles(q.reshape(-1,1), bw=.01) for q in [gq, fitgq, eq, fiteq]]

In [161]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 5.5), sharey=False)
dat_cq, fit_cq, dat_eq, fit_eq = kdefits
sns.kdeplot(fit_cq, color=greens[3], cumulative=True, ax=ax1, linestyle='--')
sns.kdeplot(fit_eq, color=reds[2], cumulative=True, ax=ax2, linestyle='--')

sns.kdeplot(dat_cq, color=greens[1], cumulative=True, ax=ax1)
sns.kdeplot(dat_eq, color=reds[-1], cumulative=True, ax=ax2)

vis.scurves([sc, fitsc], ax=ax3, linestyles=['-', '--'])

[   0.   50.  100.  150.  200.]


[150.45504550455047,
 150.45504550455047,
 149.85498549854987,
 149.85498549854987]

In [137]:
vis.scurves([y[1:6], yhat[1:6]], colors=['k', reds[1]])


[   0.   50.  100.  150.  200.]


/home/kyle/anaconda/lib/python2.7/site-packages/matplotlib/axes/_axes.py:519: UserWarning:

No labelled objects found. Use label='...' kwarg on individual plots.



[150.45504550455047,
 150.45504550455047,
 149.85498549854987,
 149.85498549854987]

In [ ]:
0.0114422

In [45]:
from radd.tools import theta
keys = np.sort(m.inits.keys())
keys = np.sort(m.pc_map.keys())
xmin, xmax = theta.format_basinhopping_bounds(keys, nlevels=2)
x = np.hstack(np.hstack([m.inits[pk] for pk in keys])).tolist()
bounds = map((lambda x: tuple([x[0], x[1]])), zip(xmin, xmax))

In [46]:
bounds

[(0.1, 5.0), (0.1, 5.0)]

In [24]:
m.opt.basins_popt

[{'a': 0.5000000316830413,
  'ssv': -0.98999998377815457,
  'tr': 0.20000000951898278,
  'v': 1.2000000443786287,
  'xb': 0.88999947113717814},
 {'a': 0.48307422187506055,
  'ssv': -0.89523520290637926,
  'tr': 0.21946076386967153,
  'v': 0.98391803843593095,
  'xb': 2.5240833660418263}]

In [23]:
m.observed_flat

[array([ 0.98347,  1.     ,  1.     ,  0.95   ,  0.7    ,  0.05   ,
         0.51138,  0.53854,  0.56498,  0.58189,  0.60502,  0.50795,
         0.53846,  0.54522,  0.55883,  0.58478]),
 array([ 0.97934,  1.     ,  1.     ,  1.     ,  0.825  ,  0.175  ,
         0.52824,  0.55255,  0.57176,  0.58522,  0.61171,  0.5291 ,
         0.54493,  0.55508,  0.57299,  0.59411]),
 array([ 0.90496,  1.     ,  1.     ,  1.     ,  0.725  ,  0.35   ,
         0.52412,  0.56516,  0.59156,  0.61174,  0.63838,  0.5183 ,
         0.55952,  0.57859,  0.59835,  0.60964])]

In [22]:
m.yhat_flat_list

[array([ 0.90439,  1.     ,  0.99999,  0.98831,  0.76266,  0.29024,
         0.53717,  0.5656 ,  0.58524,  0.60516,  0.63106,  0.52464,
         0.54929,  0.5676 ,  0.58671,  0.61199]),
 array([ 0.90424,  1.     ,  0.99999,  0.98789,  0.76363,  0.28729,
         0.53726,  0.56575,  0.58518,  0.6051 ,  0.63106,  0.52484,
         0.54979,  0.56786,  0.58698,  0.61221]),
 array([ 0.90437,  1.     ,  0.99997,  0.98817,  0.7625 ,  0.28867,
         0.53732,  0.56591,  0.58529,  0.60539,  0.63094,  0.52456,
         0.54941,  0.56782,  0.58693,  0.61249])]

## Testing custom take_step and accept_test classes (see radd>fit.py)

In [18]:
keys = np.sort(re_popt.keys())
bounder = fit.BasinBounds(keys=keys, nlevels=1)
takestep = fit.HopStep(keys, stepsize=.05)

In [39]:
x = [re_popt[k] for k in keys]
for i in range(200):
    
    inbounds=False
    
    while not inbounds:
        x = takestep.__call__(x)
        inbounds = bounder.__call__(x_new = x)
        